# =====DAMLAS ASSIGNMENT # 2=====

Data Analytics and Machine Learning at Scale
Target, Minneapolis

ASSIGNMENT #2
Version 2016-06-26
Prepared by Dr. James G. Shanahan


SCHEDULE: This Homework is due by  Friday, July 8, 2016 at 11 AM (Central Time).


=== INSTRUCTIONS for SUBMISSIONS ===
Follow the instructions for submissions carefully.

Prepare a single Jupyter note, please include questions, and question numbers in the questions and in the responses.
Submit your homework notebook via the following form:

   + http://goo.gl/forms/er3OFr5eCMWDngB72


=== Other BACKGROUND INFORMATION
This Homework, you will need to use a map-reduce framework to solve all  problems involving data. You can use MrJob or raw Hadoop mapreduce streaming.

The following notebook serves as a useful Notebook to extend for this Homework (loaded with examples and boiler plate code)

   +  https://www.dropbox.com/s/pjd6maluq4ogt7m/HW02-Supporting-Material.ipynb?dl=0

Please refer to the following slides and reference material for more background


Pattern in MapReduce : 

    + Lin, Jimmy, & Dyer, Chris. (2010). Data-intensive text processing with MapReduce. 
      San Rafael, CA: Morgan & Claypool Publishers. 
      Chapter 3
      Book available here: https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf

   + Chapter 2 in this book: Hadoop with MRJob (calculater salare average etc.) 
          -  https://www.dropbox.com/s/jd3z2s216p9kc1z/hadoop-with-python-MRJOB.pdf?dl=0
          -  Source code: https://www.dropbox.com/sh/j8oettuxbgztk0p/AAAwq9PpEeByecDmaSNslnBPa?dl=0

   + http://mrjob.readthedocs.io/en/latest/

Counters in MRJob
   + http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/5thl14n4pqvhzt5/Counter.ipynb 

KMeans Clustering (a form of Flat clustering)
   + Slides:  https://www.dropbox.com/s/6ef1mlrqr3xpnw0/Lecture07.1-Clustering.pdf?dl=0
   + http://nlp.stanford.edu/IR-book/pdf/16flat.pdf
   + http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/dcw8evd9v0su3xu/K_Means_Unit_Test_Notebook.ipynb


## HW2.0:
How do you merge  two sorted  lists/arrays of records of the form [key, value]? Where is this  used in Hadoop MapReduce? [Hint within the shuffle]
What is  a combiner function in the context of Hadoop? 
Give an example where it can be used and justify why it should be used in the context of this problem.
What is the Hadoop shuffle?


In [9]:
%%writefile MergeSortedArray.py

def mergeSort(keyValuePair1, keyValuepair2):
    sortedKeyValue=[]
    while len(keyValuePair1) and len(keyValuePair2):
        for key1 in keyValuePair1.keys():
            for key2 in keyValuePair2.keys():
                if key1 < key2:
                    sortedKeyValue[key1]=keyValuePair1
                    del keyValuePair1(key1)
                else:
                    sortedKeyValue[key2]=keyValuePair2
                    del keyValuePair1(key2)
    return  sortedKeyValue       
    

Overwriting MergeSortedArray.py


The above is used in Shuffle pahse. During the shuffle phase, MapReduce partitions data among the various reducers.
An implementation of Partitioner takes the key and value of the record, as well as the total number of reduce tasks, and returns the reduce task number that the record should go to. By default, MapReduce uses HashPartitioner, which calls hashCode() on the key and returns the result modulo of the number of reduce tasks. 

The combiner gives applications a chance to apply their reducer logic early on. As the map phase writes output, it is serialized and placed into an in-memory buffer. When this buffer fills, MapReduce will sort the buffer and then run the combiner if the application has provided an implementation for it. After the shuffle, each reducer will have one input for each map. The reducer needs to merge these inputs in order to begin processing. It is not efficient to merge too many inputs simultaneously. Thus, if the number of inputs exceeds a certain value, the data will be merged and rewritten to disk before being given to the reducer. During this merge, the combiner will be applied in an attempt to reduce the size of the input data. 



## HW2.1: Counters as a debugging aid (and for getting work done, but please use sparingly as they are heavy)

Consumer complaints dataset: Use Counters to do EDA (exploratory data analysis and to monitor progress)
Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Here’s is the first few lines of the  of the Consumer Complaints  Dataset:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

User-defined Counters

Now, let’s use MapReduce Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 

In [1]:
%%writefile MRCounter.py


from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
import ast
WORD_RE = re.compile(r"[\w']+")


class MRCounter(MRJob):
        
    def mapper(self, _, line):
        words=line.split(',')
        word=words[1]
        if word=='Debt collection':
            self.increment_counter('mapper', word, 1) 
        elif word=='Mortgage':
            self.increment_counter('mapper', word, 1)
        else:
            self.increment_counter('mapper', word, 1)

            
    def reducer(self, key, value):
        self.increment_counter('reducer', word, 1)
        
    def steps(self):
        return [MRStep(mapper=self.mapper ,reducer=self.reducer)]
        
if __name__ == '__main__':
    MRCounter.run()

Overwriting MRCounter.py


In [2]:
from MRCounter import MRCounter
mr_job = MRCounter(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    counters= runner.counters()[0]['mapper']
    del counters['Product']
    nOthers=0
    nDebt=0
    nMortgage=0
    for key, value in counters.items():
        if key=='Debt collection':
            nDebt=value
        elif key=='Mortgage':
            nMortgage=value
        else:
            nOthers+=value
    
print "Debt Collection = %d" %nDebt
print "Mortgage = %d" %nMortgage
print "Others = %d" %nOthers


Debt Collection = 44372
Mortgage = 125752
Others = 142788


## HW2.2: Analyze the performance of your Mappers, Combiners and Reducers using Counters

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux

2.2.0.0
Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

2.2.0.1
Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

2.2.0.2
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 
Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

2.2.1:  
Using 2 reducers: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). Please use a combiner.

#### Answer to Problem 2.2.0.0

In [3]:
!echo foo foo quux labs foo bar quux > WordCount.txt

In [4]:
%%writefile MRJobCounter.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 
WORD_RE = re.compile(r"[\w']+")
 
class MRWordFreqCount(MRJob):
    def mapper(self, _, line):
        self.increment_counter('group', 'Num_mapper_calls', 1)
        for word in WORD_RE.findall(line):
            yield word.lower(), 1

    def reducer(self, word, counts):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting MRJobCounter.py


In [5]:
from MRJobCounter import MRWordFreqCount
mr_job = MRWordFreqCount(args=['WordCount.txt'])
with mr_job.make_runner() as runner: 
    runner.run()
    Counters=runner.counters()[0]['group']
    for key, value in Counters.items():
        print key, value
    

Num_mapper_calls 1
Num_reducer_calls 4


#### The reason for num mapper=1 , is because the sentences are read line by line and there is only one line. The number of reducer=4 because of 4 distinct word types.

#### Answer to Problem 2.2.0.1

In [6]:
%%writefile MapperReducerComplaintCounter.py


from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
import ast


class MapperReducerComplaintCounter(MRJob):
    
    def __init__(self, *args, **kwargs):
        super(MapperReducerComplaintCounter, self).__init__(*args, **kwargs)
        self.issuesDict={}
    
    def mapper_get_lines(self, _, line):
        self.line=line
        yield self.line, 1
        self.increment_counter('group', 'mapper_get_lines', 1)
          
    
    def mapper_get_issues(self):
        complaint_id, product, sub_product, issue, _ = self.line.split(',', 4)
        if issue.startswith('"'):
            issue=issue.split('"')[1]
            issue=issue.replace(' ','_')
        if '/' in issue:
            issue=issue.replace(' ','_')
            issue=issue.replace('/','_')
        del _
        self.increment_counter('group', 'mapper_get_issues', 1)
        yield issue, 1

            
    def reducer_group_issues(self, key, value):
        val=value.next()
        self.increment_counter('group', 'reducer_group_issues', 1)
        self.issuesDict.setdefault(key, 0)
        self.issuesDict[key] += val
        
    def reducer_yield_issues_count(self):
        self.increment_counter('group', 'reducer_yield_issues_count', 1)
        for key, value in self.issuesDict.items():
            yield key, value
        
    def steps(self):
        return [MRStep(
                    mapper=self.mapper_get_lines,  # Mapper 
                    mapper_final=self.mapper_get_issues, #Mapper final
                    reducer=self.reducer_group_issues, #Reducer
                    reducer_final=self.reducer_yield_issues_count), # Reducer final
            ]
        
if __name__ == '__main__':
    MapperReducerComplaintCounter.run()

Overwriting MapperReducerComplaintCounter.py


In [8]:
from MapperReducerComplaintCounter import MapperReducerComplaintCounter
mr_job = MapperReducerComplaintCounter(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=1"'])
with mr_job.make_runner() as runner: 
    runner.run()
    for key, value in runner.counters()[0]['group'].items():
        print key, value

reducer_yield_issues_count 2
mapper_get_issues 2
mapper_get_lines 312913
reducer_group_issues 312915


#### Answer to 2.2.0.2

In [1]:
%%writefile MapperReducerComplaintCounterCombiner.py


from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
import ast

class MapperReducerComplaintCounterCombiner(MRJob):
    
    def __init__(self, *args, **kwargs):
        super(MapperReducerComplaintCounterCombiner, self).__init__(*args, **kwargs)
        self.issuesDict={}
         
    
    def mapper_get_issues(self, _, line):
        self.increment_counter('group', 'mapper_get_issues', 1)
        line = line.decode('utf8', 'ignore')
        complaint_id, complaint_type, sub_product, issue, _ = line.split(',', 4)
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        issue = regex.sub(' ', issue.lower())
        issue = re.sub( '\s+', ' ', issue)
        issue = issue.split()
        for word in issue:
            yield (word, 1)

    def reducer_yield_issues_count(self, key, value):
        self.increment_counter('group', 'reducer_yield_issues_count', 1)
        for key, value in self.issuesDict.items():
            yield key, sum(value)
        
    def steps(self):
        return [MRStep(
                    mapper=self.mapper_get_issues, #Mapper final
                    reducer=self.reducer_yield_issues_count), # Reducer final
            ]
    
    
       
if __name__ == '__main__':
    MapperReducerComplaintCounterCombiner.run()
    
    


Overwriting MapperReducerComplaintCounterCombiner.py


In [2]:
from MapperReducerComplaintCounterCombiner import MapperReducerComplaintCounterCombiner
mr_job = MapperReducerComplaintCounterCombiner(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=1"'])
with mr_job.make_runner() as runner: 
    runner.run()
    for key, value in runner.counters()[0]['group'].items():
        print key, value

mapper_get_issues 312913
reducer_yield_issues_count 172


In [4]:
!!cat Consumer_Complaints.csv | wc -l

['  312913']

#### The numbers related to following
#####  number of mappers == number of lines in the file 
#####  number of combiner == number of unique issues

#### Least 10 and top 50 counts 
 What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items).

#### 2.2.1:
Using 2 reducers: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). Please use a combiner.

In [1]:
%%writefile MapperReducerComplaintBottomN.py


from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
import ast


class MapperReducerComplaintBottomN(MRJob):
    
    def __init__(self, *args, **kwargs):
        super(MapperReducerComplaintBottomN, self).__init__(*args, **kwargs)
        self.issuesDict={}
    
    
    def mapper_get_issues(self, _, line):
        # converting bytes to utf8 string - deals non-basic/keyboard characters
        line = line.decode('utf8', 'ignore')
        
        complaint_id, complaint_type, sub_product, issue, _ = line.split(',', 4)
        
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        issue = regex.sub(' ', issue.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        issue = re.sub( '\s+', ' ', issue)
        
        # Creates a list of the words
        issue = issue.split()
        del complaint_type, sub_product, _
        
        # If it can't convert complaint id to an int, 
        # then skip that row of data (i.e., skip the headder in the csv)
        try:
            int(complaint_id)
        except: 
            return
        
        for word in issue:
            yield (word, 1)

            
    def combiner_group_issues(self, key, value):
        yield (key, sum(value))
        
    def reducer_yield_issues_count(self, key, value):
        yield (key, sum(value))
        
    def mapper_sort_issues(self, key, value):
        count = str(value).zfill(6)
        yield ((count, key), None)
        
        
    def reducer_sort_issues(self, count_word, _):
        count, word = count_word
        self.issuesDict.setdefault(word, 0)
        self.issuesDict[word]+=int(count)
        yield (word, int(count))
        
        
    def steps(self):
        return [MRStep(
                    mapper=self.mapper_get_issues, 
                    combiner=self.combiner_group_issues,
                    reducer=self.reducer_yield_issues_count),
                MRStep(
                    mapper=self.mapper_sort_issues,
                    reducer=self.reducer_sort_issues)
            ]
    
    
       
if __name__ == '__main__':
    MapperReducerComplaintTopN.run()

Overwriting MapperReducerComplaintBottomN.py


In [2]:
from MapperReducerComplaintBottomN import MapperReducerComplaintBottomN
mr_job = MapperReducerComplaintBottomN(args=['Consumer_Complaints.csv', '--jobconf="mapred.reduce.tasks=1"'])
issuesCountDict={}

with mr_job.make_runner() as runner: 
    runner.run()
    for line in runner.stream_output():
        issue, count = line.split()
        issuesCountDict.setdefault(count, [issue])
        issuesCountDict[count].append(issue)

Counts=map(int, issuesCountDict.keys())
Counts.sort()
for count in Counts[0:9]:
    print count,  set(issuesCountDict[str(count)])

64 set(['"missing"', '"disclosures"'])
71 set(['"amt"', '"day"'])
75 set(['"checks"', '"convenience"'])
92 set(['"credited"', '"payment"'])
98 set(['"amount"'])
118 set(['"apply"'])
127 set(['"overlimit"'])
131 set(['"stop"', '"charges"'])
139 set(['"receive"', '"did"', '"applied"', '"sale"'])


In [3]:
%%writefile MapperReducerComplaintTopN.py


from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
import ast


class MapperReducerComplaintTopN(MRJob):
    
    def __init__(self, *args, **kwargs):
        super(MapperReducerComplaintTopN, self).__init__(*args, **kwargs)
        self.issuesCountDict={}
    
    def configure_options(self):
        super(MapperReducerComplaintTopN, self).configure_options()
        self.add_passthrough_option(
            '--highest_n', default=50, help="Specify the top N words")
    
    def mapper_get_issues(self, _, line):
        # converting bytes to utf8 string - deals non-basic/keyboard characters
        line = line.decode('utf8', 'ignore')
        
        complaint_id, complaint_type, sub_product, issue, _ = line.split(',', 4)
        
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        issue = regex.sub(' ', issue.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        issue = re.sub( '\s+', ' ', issue)
        
        # Creates a list of the words
        issue = issue.split()
        del complaint_type, sub_product, _
        
        # If it can't convert complaint id to an int, 
        # then skip that row of data (i.e., skip the headder in the csv)
        try:
            int(complaint_id)
        except: 
            return
        
        for word in issue:
            yield (word, 1)

            
    def combiner_group_issues(self, key, value):
        yield (key, sum(value))
        
    def reducer_yield_issues_count(self, key, value):
        yield (key, sum(value))
        
    def mapper_sort_issues(self, key, value):
        count = str(value).zfill(6)
        yield ((count, key), None)
        
        
    def reducer_sort_issues(self, count_word, _):
        count, word = count_word
        self.issuesCountDict.setdefault(count, [word])
        self.issuesCountDict[count].append(word)
        
    def reducer_return_top(self):
        nTop=self.options.highest_n
        Counts=map(int, self.issuesCountDict.keys())
        Counts.sort(reverse=True)
        for count in Counts[0:nTop-1]:
            yield count,  self.issuesCountDict[str(count).zfill(6)]
        
        
    def steps(self):
        return [MRStep(
                    mapper=self.mapper_get_issues, 
                    combiner=self.combiner_group_issues,
                    reducer=self.reducer_yield_issues_count),
                MRStep(
                    mapper=self.mapper_sort_issues,
                    reducer=self.reducer_sort_issues,
                    reducer_final=self.reducer_return_top)
            ]
    
    
       
if __name__ == '__main__':
    MapperReducerComplaintTopN.run()

Overwriting MapperReducerComplaintTopN.py


In [7]:
!python MapperReducerComplaintTopN.py 'Consumer_Complaints.csv'  --jobconf=mapred.reduce.tasks=1 

No configs found; falling back on auto-configuration
Creating temp directory /tmp/MapperReducerComplaintTopN.root.20160710.060242.739207
Running step 1 of 2...
Running step 2 of 2...
Streaming final output from /tmp/MapperReducerComplaintTopN.root.20160710.060242.739207/output...
119630	["loan", "loan"]
70487	["modification", "modification"]
55251	["credit", "credit"]
36767	["servicing", "servicing"]
34903	["report", "report"]
29133	["incorrect", "incorrect"]
29069	["information", "information", "on"]
22533	["or", "or"]
20681	["account", "account"]
19309	["debt", "debt"]
16448	["and", "and"]
16205	["opening", "opening"]
12545	["club", "club", "health"]
12353	["not", "not"]
11848	["attempts", "attempts", "collect", "cont", "d", "owed"]
10885	["of", "of"]
10731	["my", "my"]
10555	["deposits", "deposits", "withdrawals"]
9484	["problems", "problems"]
8868	["application", "application"]
8401	["to", "to"]
8178	["unable", "unable"]
8158	["billing", "billing"]
7886	["other", "other"]
6938	["di

## Top 50 Terms, it's frequency and relative frequency


## HW2.3: Shopping Cart Analysis
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.
  
For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 


Do some exploratory data analysis of this dataset guided by the following questions:. 

How many unique items are available from this supplier?

Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

2.3.1 OPTIONAL 
Using 2 reducers:  Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 



In [3]:
%%writefile MRShoppingCartAnalysis.py


from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
import ast


class MRShoppingCartAnalysis(MRJob):
    
    def __init__(self, *args, **kwargs):
        super(MRShoppingCartAnalysis, self).__init__(*args, **kwargs)
        self.issuesCountDict={}
    
    def configure_options(self):
        super(MRShoppingCartAnalysis, self).configure_options()
        self.add_passthrough_option(
            '--highest_n', default=50, help="Specify the top N words")
    
    def mapper_get_issues(self, _, line):
        # converting bytes to utf8 string - deals non-basic/keyboard characters
        line = line.decode('utf8', 'ignore')
        items = line.split(' ')
        for word in set(items[:-1]): # Count Unique item in a basket
            yield (word, 1)

    def combiner_group_issues(self, key, value):
        yield (key, sum(value))
        
    def reducer_yield_issues_count(self, key, value):
        yield (key, sum(value))
        
    def mapper_sort_issues(self, key, value):
        count = str(value).zfill(6)
        yield ((count, key), None)
        
        
    def reducer_sort_issues(self, count_word, _):
        count, word = count_word
        self.issuesCountDict.setdefault(count, [word])
        self.issuesCountDict[count].append(word)
        
    def reducer_return_top(self):
        nTop=self.options.highest_n
        Counts=map(int, self.issuesCountDict.keys())
        Counts.sort(reverse=True)
        for count in Counts[0:nTop-1]:
            yield count,  self.issuesCountDict[str(count).zfill(6)]
        
        
    def steps(self):
        return [MRStep(
                    mapper=self.mapper_get_issues, 
                    combiner=self.combiner_group_issues,
                    reducer=self.reducer_yield_issues_count),
                MRStep(
                    mapper=self.mapper_sort_issues,
                    reducer=self.reducer_sort_issues,
                    reducer_final=self.reducer_return_top)
            ]
    
    
       
if __name__ == '__main__':
    MRShoppingCartAnalysis.run()

Overwriting MRShoppingCartAnalysis.py


In [4]:
from MRShoppingCartAnalysis import MRShoppingCartAnalysis
from collections import OrderedDict
import ast
mr_job = MRShoppingCartAnalysis(args=['ProductPurchaseData.txt', '--jobconf="mapred.reduce.tasks=1"'])
issuesCountDict=OrderedDict()

with mr_job.make_runner() as runner: 
    runner.run()
    for line in runner.stream_output():
        countList=line.split('\t')
        count=int(countList[0])
        products=ast.literal_eval(countList[1].split('\n')[0])
        issuesCountDict.setdefault(count, products)
        issuesCountDict[count].extend(products)

Counts=issuesCountDict.keys()
Counts.sort(reverse=True)
allItems=[]
print "Top 50 Items by Count"
for count in Counts[0:49]:
    print "Count=%d"%count,  set(issuesCountDict[count])

allItems=0
for count in Counts:
    allItems +=len(issuesCountDict[count])
    

print "Total Unique Items=%d" %allItems

Top 50 Items by Count
Count=6667 set(['DAI62779'])
Count=3881 set(['FRO40251'])
Count=3875 set(['ELE17451'])
Count=3602 set(['GRO73461'])
Count=3044 set(['SNA80324'])
Count=2851 set(['ELE32164'])
Count=2736 set(['DAI75645'])
Count=2455 set(['SNA45677'])
Count=2330 set(['FRO31317'])
Count=2293 set(['DAI85309'])
Count=2292 set(['ELE26917'])
Count=2233 set(['FRO80039'])
Count=2115 set(['GRO21487'])
Count=2083 set(['SNA99873'])
Count=2004 set(['GRO59710'])
Count=1920 set(['GRO71621'])
Count=1918 set(['FRO85978'])
Count=1840 set(['GRO30386'])
Count=1816 set(['ELE74009'])
Count=1784 set(['GRO56726'])
Count=1773 set(['DAI63921'])
Count=1756 set(['GRO46854'])
Count=1713 set(['ELE66600'])
Count=1712 set(['DAI83733'])
Count=1702 set(['FRO32293'])
Count=1697 set(['ELE66810'])
Count=1646 set(['SNA55762'])
Count=1627 set(['DAI22177'])
Count=1531 set(['FRO78087'])
Count=1516 set(['ELE99737'])
Count=1489 set(['ELE34057', 'GRO94758'])
Count=1436 set(['FRO35904'])
Count=1420 set(['FRO53271'])
Count=140

## HW2.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs

From a data mining perspective (and the aPriori algorihtm in particular), Support and Confidence are defined as follows:

       SUPPORT
       In data mining, the support value of X (where X is a collection of cooccurring items sometimes referred to as 
       an item-set. E.g., a basket or subset of a basket) with respect to T  (a transaction database where each 
       row is a transaction such as a basket of items that have been purchased)  is defined as the proportion 
       of transactions in the  database which contains  the item-set X. (a relative frequency of sorts) 

       CONFIDENCE 
       The confidence value of a rule, X ==>  Y (where X is a collection of cooccurring items and Y is generally 
       a single item. E.g., If Diapers and Beer then Cigars were also purchased), with respect to a set of transactions T, is the 
       proportion of the transactions that contains X which also contains Y. (Think of it as  tgePr(Y|X) )

       The pairs/stripes algorithm returns cooccurrence information that can be used directly to  calculate the confidence and support. 
       Note that confidence for pair X ==>  Y will  differ from the relative frequency that results from stripes when X occurs by itself in transactions.


Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support  (OPTIONAL Feel free to add in confidence level also)



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.



In [3]:
%%writefile MRShoppingCartSupportCount.py


from mrjob.job import MRJob
from mrjob.step import MRStep
import re, string
import ast, itertools

numLines=0

class MRShoppingCartSupportCount(MRJob):
    
    def __init__(self, *args, **kwargs):
        super(MRShoppingCartSupportCount, self).__init__(*args, **kwargs)
        self.issuesCountDict={}
    
    def configure_options(self):
        super(MRShoppingCartSupportCount, self).configure_options()
        self.add_passthrough_option(
            '--highest_n', default=50, help="Specify the top N words")
    
    def mapper_get_issues(self, _, line):
        # converting bytes to utf8 string - deals non-basic/keyboard characters
        global numLines
        self.increment_counter('group', 'mapper_get_issues', 1)
        items = str(line).split()
        for subset in itertools.combinations(sorted(set(items)), 2):
            yield subset, 1
        numLines +=1
#         for word in set(items[:-1]): # Count Unique item in a basket
#             yield (word, 1)

    def combiner_group_issues(self, key, value):
        self.increment_counter('group', 'combiner_get_issues', 1)
        yield (key, sum(value))
        
    def reducer_yield_issues_count(self, key, value):
        self.increment_counter('group', 'reducer_yield_issues_count', 1)
        yield (key, sum(value))
        
    def mapper_sort_issues(self, key, value):
        self.increment_counter('group', 'mapper_sort_issues', 1)
        count = str(value).zfill(6)
        yield ((count, key), None)
        
        
    def reducer_sort_issues(self, count_word, _):
        self.increment_counter('group', 'reducer_sort_issues', 1)
        count, word = count_word
        self.issuesCountDict.setdefault(count, [word])
        self.issuesCountDict[count].append(word)
        
    def reducer_return_top(self):
        self.increment_counter('group', 'reducer_return_top', 1)
        global numLines
        nTop=self.options.highest_n
        Counts=map(int, self.issuesCountDict.keys())
        Counts.sort(reverse=True)
        for count in Counts[0:nTop-1]:
            pairsList=self.issuesCountDict[str(count).zfill(6)]
            yield [list(p) for p in set(tuple(p) for p in pairsList)], (count, count*1.0/numLines)
        
        
    def steps(self):
        return [MRStep(
                    mapper=self.mapper_get_issues, 
                    combiner=self.combiner_group_issues,
                    reducer=self.reducer_yield_issues_count),
                MRStep(
                    mapper=self.mapper_sort_issues,
                    reducer=self.reducer_sort_issues,
                    reducer_final=self.reducer_return_top)
            ]
    
    
       
if __name__ == '__main__':
    MRShoppingCartSupportCount.run()

Overwriting MRShoppingCartSupportCount.py


In [6]:
!python MRShoppingCartSupportCount.py 'ProductPurchaseData.txt'  --jobconf=mapred.reduce.tasks=1 

No configs found; falling back on auto-configuration
Creating temp directory /tmp/MRShoppingCartSupportCount.root.20160711.015503.719943
Running step 1 of 2...
Counters: 2
	group
		combiner_get_issues=1026707
		mapper_get_issues=31101
Counters: 3
	group
		combiner_get_issues=1026707
		mapper_get_issues=31101
		reducer_yield_issues_count=877095
Running step 2 of 2...
Counters: 1
	group
		mapper_sort_issues=877095
Counters: 3
	group
		mapper_sort_issues=877095
		reducer_return_top=1
		reducer_sort_issues=877095
Streaming final output from /tmp/MRShoppingCartSupportCount.root.20160711.015503.719943/output...
[["DAI62779", "ELE17451"]]	[1592, 0.05118806469245362]
[["FRO40251", "SNA80324"]]	[1412, 0.04540046943828173]
[["DAI75645", "FRO40251"]]	[1254, 0.04032024693739751]
[["FRO40251", "GRO85051"]]	[1213, 0.039001961351725026]
[["DAI62779", "GRO73461"]]	[1139, 0.03662261663612103]
[["DAI75645", "SNA80324"]]	[1130, 0.03633323687341243]
[["DAI62779", "FRO40251"]]	[1070, 0.03440403845535513]
[

## HW2.5: Stripes
Repeat 2.4 using the stripes design pattern for finding cooccuring pairs (and out.

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs


# HW2.6: KMeans Clustering Tweet Dataset  [OPTIONAL]

For this problem, please refer to and borrow from the following notebook:

   http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/dcw8evd9v0su3xu/K_Means_Unit_Test_Notebook.ipynb


Here you will use a different dataset consisting of word-frequency distributions 
for 1,000 Twitter users. These Twitter users use language in very different ways,
and were classified by hand according to the criteria:

0: Human, where only basic human-human communication is observed.

1: Cyborg, where language is primarily borrowed from other sources
(e.g., jobs listings, classifieds postings, advertisements, etc...).

2: Robot, where language is formulaically derived from unrelated sources
(e.g., weather/seismology, police/fire event logs, etc...).

3: Spammer, where language is replicated to high multiplicity
(e.g., celebrity obsessions, personal promotion, etc... )

Check out the preprints of  recent research,
which spawned this dataset:

http://arxiv.org/abs/1505.04342
http://arxiv.org/abs/1508.01843

The main data lie in the accompanying file:

topUsers_Apr-Jul_2014_1000-words.txt located at:
        https://www.dropbox.com/s/6129k2urvbvobkr/topUsers_Apr-Jul_2014_1000-words.txt?dl=0

and are of the form:

USERID,CODE,TOTAL,WORD1_COUNT,WORD2_COUNT,...
.
.

where

USERID = unique user identifier
CODE = 0/1/2/3 class code
TOTAL = sum of the word counts

Using this data, you will implement a 1000-dimensional K-means algorithm in MrJob on the users
by their 1000-dimensional word stripes/vectors using several 
centroid initializations and values of K.

Note that each "point" is a user as represented by 1000 words, and that
word-frequency distributions are generally heavy-tailed power-laws
(often called Zipf distributions), and are very rare in the larger class
of discrete, random distributions. For each user you will have to normalize
by its "TOTAL" column. Try several parameterizations and initializations:

(A) K=4 uniform random centroid-distributions over the 1000 words (generate 1000 random numbers and normalize the vectors)
(B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(D) K=4 "trained" centroids, determined by the sums across the classes. Use use the 
(row-normalized) class-level aggregates as 'trained' starting centroids (i.e., the training is already done for you!).
Note that you do not have to compute the aggregated distribution or the 
class-aggregated distributions, which are rows in the auxiliary file:

topUsers_Apr-Jul_2014_1000-words_summaries.txt located at:
   https://www.dropbox.com/s/w4oklbsoqefou3b/topUsers_Apr-Jul_2014_1000-words_summaries.txt?dl=0


Row 1: Words
Row 2: Aggregated distribution across all classes
Row 3-6 class-aggregated distributions for clases 0-3
For (A),  we select 4 users randomly from a uniform distribution [1,...,1,000]
For (B), (C), and (D)  you will have to use data from the auxiliary file: 

topUsers_Apr-Jul_2014_1000-words_summaries.txt

This file contains 5 special word-frequency distributions:

(1) The 1000-user-wide aggregate, which you will perturb for initializations
in parts (B) and (C), and

(2-5) The 4 class-level aggregates for each of the user-type classes (0/1/2/3)


In parts (B) and (C), you will have to perturb the 1000-user aggregate 
(after initially normalizing by its sum, which is also provided).
So if in (B) you want to create 2 perturbations of the aggregate, start
with (1), normalize, and generate 1000 random numbers uniformly 
from the unit interval (0,1) twice (for two centroids), using:

from numpy import random
numbers = random.sample(1000)

Take these 1000 numbers and add them (component-wise) to the 1000-user aggregate,
and then renormalize to obtain one of your aggregate-perturbed initial centroids.


###################################################################################
## Geneate random initial centroids around the global aggregate
## Part (B) and (C) of this question
###################################################################################

def startCentroidsBC(k):
    counter = 0
    for line in open("topUsers_Apr-Jul_2014_1000-words_summaries.txt").readlines():
        if counter == 2:        
            data = re.split(",",line)
            globalAggregate = [float(data[i+3])/float(data[2]) for i in range(1000)]
        counter += 1
    ## perturb the global aggregate for the four initializations    
    centroids = []
    for i in range(k):
        rndpoints = random.sample(1000)
        peturpoints = [rndpoints[n]/10+globalAggregate[n] for n in range(1000)]
        centroids.append(peturpoints)
        total = 0
        for j in range(len(centroids[i])):
            total += centroids[i][j]
        for j in range(len(centroids[i])):
            centroids[i][j] = centroids[i][j]/total
    return centroids



——
For experiments A, B, C and D and iterate until a threshold (try 0.001) is reached.
After convergence, print out a summary of the classes present in each cluster.
In particular, report the composition as measured by the total
portion of each class type (0-3) contained in each cluster,
and discuss your findings and any differences in outcomes across parts A-D.